In [1]:
import math
import os
import sys
#
# Run a series of Radex models to retrieve the column density
#
# Author: Floris van der Tak , version:  06oct08 
# This version by R. Plume - May 2022

maxiter = 100
debug   = False

mole = 'HCN'   #molecule
freq = 265.9    # frequency GHz
tkin = 20.75    # Tkin (K) # CHANGE
nh2 = 1.0e5     # nH2 cm^-3
tbg = 2.73      # Tbg (K)
obs = 2.03      # Observed line intensity (K) (tmb) # CHANGE
dv =4.07       # line width km/s (fwhm) # CHANGE
bw = 0.001      # Bandwidth (GHz)
tol = 0.005     # tolerance  



radexpath = '/Users/jane/Desktop/Radex/data/'
extension = '.dat'


#make sure the file exists (else Radex bonks)
if (os.path.exists(radexpath+mole+extension)):
    print("Using data file",radexpath+mole+extension)
else:
    print("Cannot find file: ",radexpath+mole+extension)
    sys.exit()

def write_input(cdmol):
    file = open('/Users/jane/Desktop/radexHCN.inp','w')
    file.write(mole+'.dat\n')
    file.write('/Users/jane/Desktop/radexHCN.out\n')
    file.write(str(freq*(1-bw))+' '+str(freq/(1-bw))+'\n')
    file.write(str(tkin)+'\n')
    file.write('1\n')
    file.write('H2\n')
    file.write(str(nh2)+'\n')
    file.write(str(tbg)+'\n')
    file.write(str(cdmol)+'\n')
    file.write(str(dv)+'\n')
    file.write('0\n')
    file.close()

def read_radex():
    file  = open('/Users/jane/Desktop/radexHCN.out')
    lines = file.readlines()
    file.close()
    if (lines[-2].split()[-1] != '(erg/cm2/s)'):
        print("Error: Ambiguous line selection. Reduce bandwidth?")
        sys.exit()
    return float(lines[-1].split()[-2])

# Begin of main program
oflx = obs*dv
eps  = 1.0e-20
iter = 0

# Starting values of column density and fit residual
cdmol = 1e12
ratio = 0

while (ratio > (1+tol)) or (ratio < (1-tol)) :
    iter += 1
    write_input(cdmol)
    os.system('/Users/jane/Desktop/Radex/bin/radex < /Users/jane/Desktop/radexHCN.inp > /dev/null')
    mflx  = read_radex()
    if (mflx < eps):
        print("Error: Zero or negative line intensity")
        print("See radex.out for details")
        sys.exit()
    if (debug):
        print("mflx= ",mflx)
    ratio = oflx/mflx
    cdmol = cdmol * ratio
    if (iter > maxiter):
        print("Maximum number of iterations exceeded")
        ratio = 1

fmt = "The best-fit column density is %7.2e cm^-2"
print (fmt % cdmol)

Using data file /Users/jane/Desktop/Radex/data/HCN.dat
The best-fit column density is 4.32e+14 cm^-2
